# Feed-forward neural net. 

This notebook is meant to be run **online**, in google colab to take advantage of the free GPUs. If you run the data reading cell locally, it might yak stuff up in your local directory. 

Simplifications I'm making FOR NOW: 
- No punctuation
- No test data

In [ ]:
# TODO re-tokenize

In [2]:
import torch, spacy
import random
import numpy as np
from torch import nn
from tqdm.notebook import trange, tqdm
import os


from matplotlib import pyplot as plt
from torch import optim
from torch.autograd import Variable
from torch.nn.utils import clip_grad_norm_ as clip_grad_norm
from torch.nn import functional as F

seed = 811

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic=True
torch.backends.cudnn.benchmark = False
print(torch.randn(5))

tensor([-1.1484,  0.0960, -2.0328, -0.0796, -0.7443])


In [3]:
#Read in data...

!rm -rf processed_data processed_data.zip
import urllib.request
url = "https://github.com/benartuso/bias-detection/blob/main/data/processed_data.zip?raw=true"
filename, headers = urllib.request.urlretrieve(url, filename="processed_data.zip")
!unzip processed_data.zip
print("Done")

Archive:  processed_data.zip
   creating: processed_data/
  inflating: processed_data/dev_lstm.csv  
  inflating: processed_data/biased.word.dev  
  inflating: processed_data/full_dev.csv  
  inflating: processed_data/dev_ann.csv  
  inflating: processed_data/train_ann.csv  
  inflating: processed_data/biased.word.test  
  inflating: processed_data/test_ann.csv  
  inflating: processed_data/full_test.csv  
  inflating: processed_data/biased.word.train  
  inflating: processed_data/full_train.csv  
  inflating: processed_data/train_lstm.csv  
  inflating: processed_data/test_lstm.csv  
Done


In [ ]:
# !pip install transformers --quiet
# from transformers import BertTokenizer
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


     |████████████████████████████████| 1.4MB 17.6MB/s 
     |████████████████████████████████| 2.9MB 53.8MB/s 
     |████████████████████████████████| 890kB 48.2MB/s 


In [4]:
from torchtext.data import Field, ReversibleField, Dataset, TabularDataset, BucketIterator, Iterator
from spacy.tokenizer import Tokenizer

spacy_en = spacy.load('en')

# def tokenize_fn(text):
#   return text.strip().split()

def reader(train_name, dev_name, test_name, suffix = ".csv", rpath="processed_data", batch_size=8, min_freq=2):
  #Text field, sequence data
  TXT = Field(sequential=True, tokenize="spacy", init_token=None, eos_token=None, lower=True)
  #Binary labels
  LABEL = Field(sequential=False, unk_token=None, dtype=torch.long, use_vocab=False)

  #Create Dataset
  fields = [("text", TXT), ("label", LABEL)]
  trn_data = TabularDataset(os.path.join(rpath, train_name+suffix), format="CSV", fields=fields, skip_header=True)
  val_data = TabularDataset(os.path.join(rpath, dev_name+suffix), format="CSV", fields=fields, skip_header=True)
  test_data = TabularDataset(os.path.join(rpath, test_name+suffix), format="CSV", fields=fields, skip_header=True)



  #Split
  #Build vocab on train set  
  TXT.build_vocab(trn_data, min_freq=min_freq)

  train_iter, val_iter, test_iter = BucketIterator.splits((trn_data, val_data, test_data), 
                                               batch_size=batch_size, 
                                               sort_key= lambda x: len(x.text),
                                               shuffle=False,
                                               repeat=False)
  
  return train_iter, val_iter, test_iter, TXT

In [8]:
train_iter, val_iter, test_iter, txtfield = reader("train_word", "dev_word", "test_word", suffix=".csv", min_freq=5, batch_size=16)
vocab_size = len(txtfield.vocab)

print("Vocab size = {}".format(vocab_size))
pad = txtfield.vocab.stoi[txtfield.pad_token]

print("[TRAIN]:%d (dataset:%d)\t[VAL]:%d (dataset:%d)\t"
    % (len(train_iter), len(train_iter.dataset),
    len(val_iter), len(val_iter.dataset)))
print("[vocab]:%d" % (vocab_size))

Vocab size = 27553
[TRAIN]:6726 (dataset:107606)	[VAL]:88 (dataset:1400)	
[vocab]:27553


In [9]:
txtfield.vocab.load_vectors('glove.6B.50d')

In [10]:
txtfield.vocab.vectors.shape

torch.Size([27553, 50])

In [ ]:
# # https://radimrehurek.com/gensim/models/word2vec.html#pretrained-models

# import gensim.downloader
# print(*gensim.downloader.info()['models'].keys(), sep="\n")

fasttext-wiki-news-subwords-300
conceptnet-numberbatch-17-06-300
word2vec-ruscorpora-300
word2vec-google-news-300
glove-wiki-gigaword-50
glove-wiki-gigaword-100
glove-wiki-gigaword-200
glove-wiki-gigaword-300
glove-twitter-25
glove-twitter-50
glove-twitter-100
glove-twitter-200
__testing_word2vec-matrix-synopsis


In [ ]:
# glove_vectors = gensim.downloader.load("glove-wiki-gigaword-50")

[==================================================] 100.0% 66.0/66.0MB downloaded


In [ ]:
# glove_vocab = glove_vectors.vocab.keys()

In [ ]:
# glove_vectors.word_vec(".")

array([ 1.5164e-01,  3.0177e-01, -1.6763e-01,  1.7684e-01,  3.1719e-01,
        3.3973e-01, -4.3478e-01, -3.1086e-01, -4.4999e-01, -2.9486e-01,
        1.6608e-01,  1.1963e-01, -4.1328e-01, -4.2353e-01,  5.9868e-01,
        2.8825e-01, -1.1547e-01, -4.1848e-02, -6.7989e-01, -2.5063e-01,
        1.8472e-01,  8.6876e-02,  4.6582e-01,  1.5035e-02,  4.3474e-02,
       -1.4671e+00, -3.0384e-01, -2.3441e-02,  3.0589e-01, -2.1785e-01,
        3.7460e+00,  4.2284e-03, -1.8436e-01, -4.6209e-01,  9.8329e-02,
       -1.1907e-01,  2.3919e-01,  1.1610e-01,  4.1705e-01,  5.6763e-02,
       -6.3681e-05,  6.8987e-02,  8.7939e-02, -1.0285e-01, -1.3931e-01,
        2.2314e-01, -8.0803e-02, -3.5652e-01,  1.6413e-02,  1.0216e-01],
      dtype=float32)

## Basic FFNN + CNN

In [11]:
class NeuralClassifier(nn.Module):
    def __init__(self, vocab, vocab_size, embed_size, drop_rate=0.0, class_size=2, pad=1):
        super(NeuralClassifier, self).__init__()
        """ Initialization
        - vocab_size
        - embed_size: word embedding size
        - drop_rate: dropout rate
        - class_size: number of classes. For binary classification, class_size = 2
        """
        # ---------------------------------
        # Configuration
        self.vocab_size = vocab_size # size of the vocab
        self.class_size = class_size # number of classes
        # self.using_cuda = True
        self.dropout = nn.Dropout(drop_rate)
        self.pad = pad
        # ---------------------------------
        # Network parameters
        # vocab.vectors = vocab.vectors.to("cuda:0")
        self.embed = nn.Embedding.from_pretrained(vocab.vectors, padding_idx=pad)
        self.fc = nn.Linear(embed_size, class_size, bias=True)


    def forward(self, batch):
        """ Forward function
        """
        input, label = batch.text, batch.label
        if torch.cuda.is_available():
          input = input.to("cuda:0")
          label = lable.to("cuda:0")
        
        # ---------------------------------
        # === Hidden layer ===
        # Sum over all the embeddings for each input text
        #   then, pass through the nonlinear Sigmoid function
        x = self.embed(input) # Dim: L x B x E
        x = self.dropout(x)
        hidden = torch.sigmoid(x.sum(axis=0)) # Dim: B x E
        
        # ---------------------------------
        # === Classification layer ===
        logit = self.fc(hidden) # Dim: Batch_size x Class_size
        # Normalization
        logprob = F.log_softmax(logit, dim=1)
        
        # ---------------------------------
        # === Loss function ===
        # Compute negative log-likelihood loss
        loss = F.cross_entropy(logprob, label)
        return loss, logprob


In [12]:
class SimpleCNN(nn.Module):
    def __init__(self, vocab, vocab_size, embed_size, drop_rate=0.0,
                 class_size=2, kernel_sizes=[2,3,4],
                 dropout=0.0, using_cuda=False, pad=None):
        super(SimpleCNN, self).__init__()
        # ---------------------------------
        # Configuration
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.class_size = class_size # number of classes
        self.kernel_sizes = kernel_sizes # a list of kernel sizes
        self.using_cuda = using_cuda
        self.pad = pad # index of <pad> in vocab

        # not sure if should include
        self.dropout = nn.Dropout(drop_rate)
        # ---------------------------------
        # Model Arch
        self.embed = nn.Embedding.from_pretrained(vocab.vectors, padding_idx=pad)
        self.conv2 = torch.nn.Conv1d(in_channels=embed_size, out_channels=embed_size, kernel_size=self.kernel_sizes[0])
        self.conv3 = torch.nn.Conv1d(in_channels=embed_size, out_channels=embed_size, kernel_size=self.kernel_sizes[1])
        self.conv4 = torch.nn.Conv1d(in_channels=embed_size, out_channels=embed_size, kernel_size=self.kernel_sizes[2])
        self.tanh = torch.tanh
        self.maxpool = torch.max
        self.fc = nn.Linear(embed_size*3, class_size, bias=True)


    def forward(self, batch):
        # ===============================
        # Implement the CNN specified in the homework
        input, label = batch.text, batch.label
        if torch.cuda.is_available():
          input = input.to("cuda:0")
          label = input.to("cuda:0")

        # initial input
        x0 = self.embed(input)
        x0 = self.dropout(x0) # L X B X E
        x0 = torch.transpose(x0, 0, 2) # -> E X B X L
        x0 = torch.transpose(x0, 0, 1) # -> B X E X L

        # through convolutional layer
        x1_0 = self.conv2(x0)
        x1_1 = self.conv3(x0)
        x1_2 = self.conv3(x0)

        # tanh activation function
        x2_0 = self.tanh(x1_0)
        x2_1 = self.tanh(x1_1)
        x2_2 = self.tanh(x1_2)

        # max pooling
        x3_0 = self.maxpool(x2_0, dim=2).values # dim=2 is L dimension, -> B X E
        x3_1 = self.maxpool(x2_1, dim=2).values
        x3_2 = self.maxpool(x2_2, dim=2).values

        # concatenating/preparing for softmax
        x4 = torch.cat([x3_0, x3_1, x3_2], dim=1) # -> B X 3E
        logit = self.fc(x4) # B X class_size

        # ===============================
        
        logprob = F.log_softmax(logit, dim=1)
        loss = F.cross_entropy(logprob, label)
        return loss, logprob

In [13]:

def batch_train(batch, model, optimizer):
    """ Training with one batch
    - batch: a min-batch of the data
    - model: the defined neural network
    - optimizer: optimization method used to update the parameters
    """
    # set in training mode
    model.train()
    # initialize optimizer
    optimizer.zero_grad()
    # forward: prediction
    loss, _ = model(batch)
    # backward: gradient computation
    loss.backward()
    # norm clipping, in case the gradient norm is too large
    clip_grad_norm(model.parameters(), grad_clip)
    # gradient-based update parameter
    optimizer.step()
    return model, loss.item()

In [27]:
from sklearn.metrics import confusion_matrix

In [63]:
def eval(data_iter, model):
    """ Evaluate the model with the data
    data_iter: the data iterator 
    model: the defined model
    """
    # set in the eval model, which will trun off the features only used for training, such as droput
    model.eval()
    # records
    val_loss, val_batch = 0, 0
    total_example, correct_pred = 0, 0
    true_pos, pred_pos, label_pos = 0, 0, 0
    # iterate all the mini batches for evaluation
    for b, batch in enumerate(data_iter):
        # Forward: prediction
        loss, logprob = model(batch)
        # 
        val_batch += 1
        val_loss += loss
        # Argmax
        max_logprob, pred_label = torch.max(logprob, -1)
        if torch.cuda.is_available():
          batch = batch.label.to("cuda:0")
          

        correct_pred += (pred_label==batch.label).sum()
        total_example += batch.label.size()[0]

        # precision + recall
        correct = pred_label==batch.label
        true_pos += (correct & (pred_label == 1)).sum()
        pred_pos += pred_label.sum()
        label_pos += batch.label.sum()


    precision = 1.0*true_pos / pred_pos # truepos / (truepos + falsepos)

    recall = 1.0*true_pos / label_pos # truepos / (truepos + falseneg)


    acc = (1.0*correct_pred)/total_example
    # print("val_batch = {}".format(val_batch))
    return (val_loss/val_batch), acc, precision, recall

In [15]:
if torch.cuda.is_available():
  torch.cuda.set_device("cuda:0")


In [16]:
txtfield.vocab.stoi["<pad>"]

1

### Simple FFN training

In [24]:
model = NeuralClassifier(txtfield.vocab, vocab_size, embed_size=50, drop_rate=0.5, class_size=2, pad=1)
if torch.cuda.is_available():
  model.cuda()
optimizer = optim.SGD(model.parameters(), lr=0.001, weight_decay=0)
# the norm of grad clipping
grad_clip = 1.0

# ------------------------------------
# 3. Define the numbers of training epochs and validation steps
epoch, val_step = 10, 1000

# ------------------------------------
# 4. Training iterations
TrnLoss, ValLoss, ValAcc = [], [], []
total_batch = 0
for e in trange(epoch):
    # print(e)
    for b, batch in tqdm(enumerate(train_iter)):
        total_batch += 1
        # Update parameters with one batch
        model, loss = batch_train(batch, model, optimizer)
        # Compute validation loss after each val_step
        if total_batch % val_step == 0:
            val_loss, val_acc, _, _ = eval(val_iter, model)
            ValLoss.append(val_loss)
            ValAcc.append(val_acc)
            print(val_acc.item(), val_loss.item(), loss)
            TrnLoss.append(loss)
print("The best validation accuracy = {:.4}".format(max(ValAcc)))

0.506428599357605 0.6990084052085876 0.7142407894134521
0.5007143020629883 0.6980588436126709 0.6931551694869995
0.506428599357605 0.6969150900840759 0.7045146822929382
0.5092856884002686 0.69611656665802 0.6787758469581604
0.5014285445213318 0.6960387825965881 0.7365884780883789
0.506428599357605 0.6953698396682739 0.7055896520614624



0.5078571438789368 0.6943966150283813 0.719760000705719
0.5078571438789368 0.6939769983291626 0.66512531042099
0.5035714507102966 0.6935633420944214 0.6641715168952942
0.506428599357605 0.6931730508804321 0.7222806811332703
0.5178571343421936 0.6929954290390015 0.6848297119140625
0.5164285898208618 0.692722499370575 0.6733797192573547
0.5092856884002686 0.6925379037857056 0.6986721754074097



0.5114285945892334 0.6926526427268982 0.7133064866065979
0.5142857432365417 0.69206303358078 0.6790033578872681
0.5199999809265137 0.6919512748718262 0.679776668548584
0.5242857336997986 0.6917044520378113 0.7102633714675903
0.5199999809265137 0.6917334198951721 0.696571409702301
0.5257142782211304 0.6915209293365479 0.6972192525863647
0.5221428275108337 0.6915174126625061 0.680397093296051



0.522857129573822 0.6914070248603821 0.6517410278320312
0.522857129573822 0.6912634372711182 0.6851622462272644
0.5214285850524902 0.6911830306053162 0.6962278485298157
0.5257142782211304 0.6911265254020691 0.6969754695892334
0.5192857384681702 0.6910762786865234 0.6557977199554443
0.5249999761581421 0.6910758018493652 0.7065985202789307



0.5221428275108337 0.6912274956703186 0.6726487278938293
0.5264285802841187 0.6909794807434082 0.7006804943084717
0.5278571248054504 0.6908920407295227 0.6755104064941406
0.5264285802841187 0.6908526420593262 0.6896653175354004
0.5264285802841187 0.6907957792282104 0.7117125988006592
0.5350000262260437 0.691043496131897 0.713593602180481
0.5257142782211304 0.6908503770828247 0.6985193490982056



0.5249999761581421 0.6907268166542053 0.6986274719238281
0.5299999713897705 0.6906958222389221 0.6836580634117126
0.5314285755157471 0.6906402111053467 0.6979212164878845
0.529285728931427 0.6906185150146484 0.6847871541976929
0.5285714268684387 0.6906419992446899 0.7066100239753723
0.5314285755157471 0.6906941533088684 0.7143804430961609
0.5328571200370789 0.6906303763389587 0.7068316340446472



0.5278571248054504 0.6907602548599243 0.6777967214584351
0.5307142734527588 0.6908426880836487 0.6972967982292175
0.5299999713897705 0.6906372308731079 0.6897752285003662
0.5335714221000671 0.6905133128166199 0.6990770101547241
0.5235714316368103 0.6907312870025635 0.7265588641166687
0.5357142686843872 0.6907187104225159 0.6820961236953735
0.529285728931427 0.6905374526977539 0.6634547710418701



0.5257142782211304 0.6906630992889404 0.6768369078636169
0.5278571248054504 0.690742015838623 0.7220885753631592
0.543571412563324 0.6904507279396057 0.6991515159606934
0.5357142686843872 0.6907325983047485 0.6788560748100281
0.5314285755157471 0.6905523538589478 0.7114880681037903
0.5335714221000671 0.6906643509864807 0.709006130695343



0.5328571200370789 0.6904574036598206 0.69842129945755
0.5428571701049805 0.6904368996620178 0.682686448097229
0.5350000262260437 0.6904134154319763 0.6861550807952881
0.5378571152687073 0.6903978586196899 0.7030956745147705
0.5328571200370789 0.6905915141105652 0.7102224826812744
0.5321428775787354 0.6909077763557434 0.7237015962600708
0.5350000262260437 0.6904438734054565 0.6845921277999878



0.5342857241630554 0.6904050707817078 0.6926212310791016
0.5321428775787354 0.6905855536460876 0.6901772618293762
0.5400000214576721 0.6904016137123108 0.6951757073402405
0.5342857241630554 0.6903519034385681 0.6907908320426941
0.5307142734527588 0.6904684901237488 0.6772621273994446
0.5328571200370789 0.6905645728111267 0.691306471824646
0.5342857241630554 0.6903717517852783 0.6891876459121704


The best validation accuracy = 0.5436


In [64]:
test_loss, test_acc, test_prec, test_rec = eval(test_iter, model)
test_acc.item()

0.5354999899864197

In [65]:
test_prec.item(), test_rec.item()

(0.5347018837928772, 0.546999990940094)

### CNN training

In [17]:
cnn = SimpleCNN(txtfield.vocab, vocab_size, embed_size=50, drop_rate=0.5, class_size=2, pad=1)
if torch.cuda.is_available():
  cnn.cuda()
optimizer_cnn = optim.SGD(cnn.parameters(), lr=0.001, weight_decay=0)#, momentum=0.9)
# the norm of grad clipping
grad_clip = 1.0

# ------------------------------------
# 3. Define the numbers of training epochs and validation steps
epoch, val_step = 10, 1000

# ------------------------------------
# 4. Training iterations
TrnLoss_cnn, ValLoss_cnn, ValAcc_cnn = [], [], []
total_batch = 0
for e in trange(epoch):
    # print(e)
    for b, batch in tqdm(enumerate(train_iter)):
        total_batch += 1
        # Update parameters with one batch
        cnn, loss = batch_train(batch, cnn, optimizer_cnn)
        # Compute validation loss after each val_step
        if total_batch % val_step == 0:
            val_loss, val_acc, _, _ = eval(val_iter, cnn)
            ValLoss_cnn.append(val_loss.item())
            ValAcc_cnn.append(val_acc.item())
            print(val_acc.item(), val_loss.item(), loss)
            TrnLoss_cnn.append(loss)
print("The best validation accuracy = {:.4}".format(max(ValAcc_cnn)))

0.48571428656578064 0.6958437561988831 0.6947653889656067
0.5028571486473083 0.6957464814186096 0.6811731457710266
0.5057142972946167 0.694928765296936 0.7056794762611389
0.4950000047683716 0.6943091154098511 0.6767168045043945
0.5057142972946167 0.6979855298995972 0.6785067319869995
0.5035714507102966 0.6981737613677979 0.6776710152626038



0.5021428465843201 0.6955167055130005 0.6796099543571472
0.5107142925262451 0.6938474774360657 0.7127534747123718
0.506428599357605 0.6955003142356873 0.707672655582428
0.5071428418159485 0.6957998871803284 0.7233219146728516
0.5099999904632568 0.6947481632232666 0.6957641839981079
0.5114285945892334 0.6933875679969788 0.6742205619812012
0.5157142877578735 0.6932817697525024 0.707053542137146



0.5149999856948853 0.6920918226242065 0.6590741276741028
0.5192857384681702 0.6932194828987122 0.6836702227592468
0.5042856931686401 0.6960693597793579 0.6875925660133362
0.5142857432365417 0.6932748556137085 0.7233386635780334
0.5114285945892334 0.6933218836784363 0.6847847104072571
0.5099999904632568 0.6937539577484131 0.6586854457855225
0.5214285850524902 0.69204181432724 0.6864569187164307



0.508571445941925 0.6945218443870544 0.6901708245277405
0.5107142925262451 0.6933832764625549 0.701410174369812
0.5178571343421936 0.6925061345100403 0.6925899386405945
0.506428599357605 0.6958574652671814 0.7027180790901184
0.5107142925262451 0.6933924555778503 0.7115621566772461
0.5142857432365417 0.6932462453842163 0.6760196685791016



0.5214285850524902 0.6908268332481384 0.6816182732582092
0.5171428322792053 0.6911935210227966 0.6666867136955261
0.5192857384681702 0.6914988160133362 0.6640418767929077
0.5214285850524902 0.692184567451477 0.7229695916175842
0.5142857432365417 0.6931445598602295 0.6911904215812683
0.5171428322792053 0.6923100352287292 0.6917771100997925
0.5192857384681702 0.6926074624061584 0.6940090656280518



0.5185714364051819 0.6916535496711731 0.689664363861084
0.5185714364051819 0.6916182637214661 0.6983418464660645
0.5185714364051819 0.6910626888275146 0.6945820450782776
0.5299999713897705 0.6896920800209045 0.6744424700737
0.5414285659790039 0.688707172870636 0.6991092562675476
0.5307142734527588 0.6895824074745178 0.6894164681434631
0.529285728931427 0.6892569661140442 0.7079870700836182



0.5421428680419922 0.6882995367050171 0.6790242195129395
0.5392857193946838 0.6875254511833191 0.7084435224533081
0.5264285802841187 0.6907896399497986 0.7116649150848389
0.5364285707473755 0.6885485053062439 0.6831397414207458
0.5364285707473755 0.6875464916229248 0.673992931842804
0.5307142734527588 0.6894060969352722 0.6963460445404053
0.5392857193946838 0.6875743269920349 0.6937743425369263



0.5350000262260437 0.6865597367286682 0.6547895669937134
0.5278571248054504 0.6897998452186584 0.6804797649383545
0.5414285659790039 0.6875674724578857 0.7391947507858276
0.5414285659790039 0.6875233054161072 0.6545425057411194
0.5400000214576721 0.6872091889381409 0.6621604561805725
0.5385714173316956 0.6877161264419556 0.7072098851203918



0.5392857193946838 0.6867172718048096 0.7135716080665588
0.5407142639160156 0.6874485611915588 0.6853732466697693
0.5400000214576721 0.6852375864982605 0.6888505816459656
0.5350000262260437 0.6841617822647095 0.6986536979675293
0.545714259147644 0.6867465972900391 0.6935911178588867
0.5421428680419922 0.6863254308700562 0.6969723105430603
0.543571412563324 0.6852183938026428 0.6692502498626709



0.5528571605682373 0.6825940608978271 0.7008376121520996
0.5471428632736206 0.6852774620056152 0.683374285697937
0.5464285612106323 0.6838167309761047 0.6740930080413818
0.5642856955528259 0.6808610558509827 0.6869471669197083
0.5657142996788025 0.6804773807525635 0.7268858551979065
0.5478571653366089 0.6828106641769409 0.7014477252960205
0.566428542137146 0.6802371144294739 0.6888116598129272


The best validation accuracy = 0.5664


In [18]:
test_loss, test_acc, _, _ = eval(test_iter, cnn)
test_acc.item()

0.5634999871253967

In [19]:
total_batch = 0
for e in trange(epoch):
    # print(e)
    for b, batch in tqdm(enumerate(train_iter)):
        total_batch += 1
        # Update parameters with one batch
        cnn, loss = batch_train(batch, cnn, optimizer_cnn)
        # Compute validation loss after each val_step
        if total_batch % val_step == 0:
            val_loss, val_acc, _, _ = eval(val_iter, cnn)
            ValLoss_cnn.append(val_loss.item())
            ValAcc_cnn.append(val_acc.item())
            print(val_acc.item(), val_loss.item(), loss)
            TrnLoss_cnn.append(loss)
print("The best validation accuracy = {:.4}".format(max(ValAcc_cnn)))

0.5657142996788025 0.6798415780067444 0.683015763759613
0.566428542137146 0.6806541085243225 0.7195122838020325
0.5699999928474426 0.6794028878211975 0.6795222163200378
0.5742856860160828 0.6785222291946411 0.6693077087402344
0.5514285564422607 0.6830376386642456 0.6793009638786316
0.5571428537368774 0.6823262572288513 0.6678861379623413



0.5678571462631226 0.6795714497566223 0.6795390844345093
0.5707142949104309 0.6777844429016113 0.7188555002212524
0.5699999928474426 0.6785061359405518 0.663106381893158
0.5699999928474426 0.6784641742706299 0.7338131070137024
0.5678571462631226 0.6777799129486084 0.6459401249885559
0.579285740852356 0.6766777038574219 0.6436322331428528
0.5714285969734192 0.6765865087509155 0.6718912124633789



0.5757142901420593 0.6756859421730042 0.6452112793922424
0.5785714387893677 0.6761003136634827 0.6913639903068542
0.566428542137146 0.6781550049781799 0.6650570034980774
0.581428587436676 0.6758633255958557 0.7280946373939514
0.5764285922050476 0.675355851650238 0.7216118574142456
0.5785714387893677 0.6755922436714172 0.618595540523529
0.5764285922050476 0.6746521592140198 0.7254946231842041



0.5771428346633911 0.6755732893943787 0.7378432750701904
0.5771428346633911 0.6748800873756409 0.6853780150413513
0.5764285922050476 0.6744868755340576 0.6731354594230652
0.5778571367263794 0.6754372119903564 0.6804404258728027
0.5771428346633911 0.6746101975440979 0.6879252791404724
0.5771428346633911 0.6742208003997803 0.664489209651947



0.5799999833106995 0.6736642718315125 0.6682738065719604
0.5799999833106995 0.6735295653343201 0.7159702181816101
0.5764285922050476 0.6732391715049744 0.687505841255188
0.5764285922050476 0.6735867261886597 0.7626769542694092
0.579285740852356 0.6734502911567688 0.7272007465362549
0.5778571367263794 0.6735393404960632 0.7232089042663574
0.5799999833106995 0.6734811663627625 0.7475153207778931



0.581428587436676 0.6730261445045471 0.6712698340415955
0.5821428298950195 0.6727921366691589 0.6436492204666138
0.5821428298950195 0.6724932193756104 0.6861331462860107
0.5807142853736877 0.6723024845123291 0.6730911135673523
0.5799999833106995 0.6724042892456055 0.7090352177619934
0.5871428847312927 0.6720309257507324 0.6928654909133911
0.581428587436676 0.6717525720596313 0.6635627746582031



0.5807142853736877 0.6717206239700317 0.6849820613861084
0.581428587436676 0.6714831590652466 0.7525056600570679
0.5878571271896362 0.672532856464386 0.6690541505813599
0.5835714340209961 0.6716046333312988 0.6801912188529968
0.5821428298950195 0.6711583733558655 0.6804033517837524
0.5807142853736877 0.6716130375862122 0.7183597683906555
0.5835714340209961 0.6710212826728821 0.677102267742157



0.5821428298950195 0.6709263324737549 0.651270866394043
0.5907142758369446 0.6719369888305664 0.6697861552238464
0.581428587436676 0.6711304783821106 0.7466685771942139
0.5835714340209961 0.6713720560073853 0.6316346526145935
0.5899999737739563 0.6712488532066345 0.6725292801856995
0.5842857360839844 0.6713280081748962 0.7013586759567261



0.5857142806053162 0.6709277629852295 0.6724758148193359
0.5864285826683044 0.6708545088768005 0.6969475746154785
0.5914285778999329 0.6703172922134399 0.7326611876487732
0.5878571271896362 0.6702033281326294 0.6967992782592773
0.5871428847312927 0.670840322971344 0.7133543491363525
0.5878571271896362 0.6709524393081665 0.684270441532135
0.5921428799629211 0.6704167723655701 0.695349931716919



0.5885714292526245 0.6705502867698669 0.6740758419036865
0.5914285778999329 0.6707181930541992 0.6888617277145386
0.5892857313156128 0.6701416373252869 0.6347017288208008
0.5928571224212646 0.669989824295044 0.6746634244918823
0.5892857313156128 0.6705504059791565 0.755510687828064
0.5878571271896362 0.6702819466590881 0.6613550186157227
0.5928571224212646 0.6698882579803467 0.7070367932319641


The best validation accuracy = 0.5929


In [20]:
test_loss, test_acc, _, _ = eval(test_iter, cnn)
test_acc.item()

0.5879999995231628

In [22]:
total_batch = 0
for e in trange(epoch):
    # print(e)
    for b, batch in tqdm(enumerate(train_iter)):
        total_batch += 1
        # Update parameters with one batch
        cnn, loss = batch_train(batch, cnn, optimizer_cnn)
        # Compute validation loss after each val_step
        if total_batch % val_step == 0:
            val_loss, val_acc, _, _ = eval(val_iter, cnn)
            ValLoss_cnn.append(val_loss.item())
            ValAcc_cnn.append(val_acc.item())
            print(val_acc.item(), val_loss.item(), loss)
            TrnLoss_cnn.append(loss)
print("The best validation accuracy = {:.4}".format(max(ValAcc_cnn)))

0.5950000286102295 0.6697684526443481 0.7316750288009644
0.5899999737739563 0.6698915958404541 0.6952908635139465
0.595714271068573 0.6701014041900635 0.6335821151733398
0.5899999737739563 0.6702708005905151 0.6735354065895081
0.5864285826683044 0.6713294386863708 0.6746456027030945
0.5857142806053162 0.6712426543235779 0.6442988514900208



0.5942857265472412 0.6699079871177673 0.6576512455940247
0.5964285731315613 0.670120120048523 0.7503186464309692
0.595714271068573 0.6697397232055664 0.6888771057128906
0.5978571176528931 0.6698962450027466 0.7587302923202515
0.5992857217788696 0.6697182059288025 0.6162736415863037
0.595714271068573 0.6692681312561035 0.6470499634742737
0.5942857265472412 0.6693997979164124 0.6987581253051758



0.5857142806053162 0.6697365641593933 0.6476609706878662
0.5964285731315613 0.669167697429657 0.7617781162261963
0.5907142758369446 0.6698788404464722 0.6515634059906006
0.5935714244842529 0.6693910360336304 0.7201611995697021
0.5921428799629211 0.6701704859733582 0.6577379703521729
0.5928571224212646 0.6696614623069763 0.6627035140991211
0.5857142806053162 0.6705083847045898 0.6782926321029663



0.5950000286102295 0.6696291565895081 0.762231707572937
0.5942857265472412 0.6699405908584595 0.6714263558387756
0.5935714244842529 0.6694274544715881 0.689034640789032
0.5942857265472412 0.669492244720459 0.72651606798172
0.5964285731315613 0.6699270606040955 0.7223189473152161
0.5935714244842529 0.6693940162658691 0.6664183139801025



0.5821428298950195 0.6698052883148193 0.6672261953353882
0.5907142758369446 0.6689233183860779 0.6831464171409607
0.5935714244842529 0.66869056224823 0.7131457328796387
0.5942857265472412 0.6685887575149536 0.7246383428573608
0.5978571176528931 0.6685405969619751 0.7048100233078003
0.5942857265472412 0.6685405373573303 0.7688626050949097
0.5942857265472412 0.6683689951896667 0.7101373076438904



0.595714271068573 0.6683762073516846 0.6736567616462708
0.5964285731315613 0.6684785485267639 0.6677019596099854
0.5992857217788696 0.66856849193573 0.7069661021232605
0.5892857313156128 0.6690818071365356 0.6486978530883789
0.5835714340209961 0.6696368455886841 0.6733425855636597
0.5964285731315613 0.6688863635063171 0.686002790927887
0.5971428751945496 0.6684894561767578 0.6627981662750244



0.5871428847312927 0.66893070936203 0.6654305458068848
0.5871428847312927 0.668997585773468 0.7560892105102539
0.5971428751945496 0.6682833433151245 0.660975456237793
0.5942857265472412 0.6687358021736145 0.7355009913444519
0.5878571271896362 0.6689154505729675 0.6601836085319519
0.5985714197158813 0.6689367890357971 0.7090228199958801
0.5907142758369446 0.6690905094146729 0.6299184560775757



0.5885714292526245 0.6692208051681519 0.644216775894165
0.5971428751945496 0.6683582663536072 0.6948109865188599
0.5985714197158813 0.6683195233345032 0.7370805144309998
0.5985714197158813 0.6682998538017273 0.6335105299949646
0.595714271068573 0.6685144901275635 0.7666468024253845
0.5978571176528931 0.6681978106498718 0.6693861484527588



0.5950000286102295 0.6681652665138245 0.6529626846313477
0.5978571176528931 0.6682267189025879 0.6926517486572266
0.5950000286102295 0.6681103706359863 0.6513872742652893
0.5935714244842529 0.6689736247062683 0.7626523375511169
0.5942857265472412 0.6684679388999939 0.72571861743927
0.5928571224212646 0.6680982112884521 0.6742309331893921
0.5985714197158813 0.6679558157920837 0.6819541454315186



0.5935714244842529 0.6679061651229858 0.6633769869804382
0.5985714197158813 0.6678960919380188 0.6971479654312134
0.5964285731315613 0.6674986481666565 0.6178891062736511
0.5935714244842529 0.6676687598228455 0.6572670340538025
0.5899999737739563 0.6683416962623596 0.7754208445549011
0.6000000238418579 0.6677699685096741 0.7255924344062805
0.5914285778999329 0.6681568622589111 0.7289696931838989


The best validation accuracy = 0.6


In [66]:
test_loss, test_acc, test_prec, test_rec = eval(test_iter, cnn)
test_acc.item()

0.5929999947547913

In [67]:
test_prec.item(), test_rec.item()

(0.5981012582778931, 0.5669999718666077)